# Tarefa 03

- Leia os enunciados com atenção
- Saiba que pode haver mais de uma resposta correta
- Insira novas células de código sempre que achar necessário
- Em caso de dúvidas, procure os Tutores
- Divirta-se :)

In [1]:
import pandas as pd
import requests

####  1) Lendo de APIs
Vimos em aula como carregar dados públicos do governo através de um API (*Application Programming Interface*). No exemplo de aula, baixamos os dados de pedidos de verificação de limites (PVL) realizados por estados, e selecionamos apenas aqueles referentes ao estado de São Paulo.

1. Repita os mesmos passos feitos em aula, mas selecione os PVLs realizados por municípios no estado do Rio de Janeiro.
2. Quais são os três *status* das solicitações mais frequentes na base? Quais são suas frequências?
3. Construa uma nova variável que contenha o ano do **status**. Observe que ```data_status``` vem como tipo *object* no **DataFrame**. Dica: você pode usar o método ```.str``` para transformar o tipo da variável em string, em seguida um método como [**slice()**](https://pandas.pydata.org/docs/reference/api/pandas.Series.str.slice.html) ou [**split()**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.split.html).
4. Indique a frequência de cada ano do campo construído no item (3).

In [35]:
# 1) Seu código aqui

# recebendo url dos dados na variavel url
url = "https://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf=RJ&tipo_interessado=Estado"

# importando os dados para o python
pvl_rj = requests.get(url)
pvl_rj.status_code

200

In [94]:
# convertendo os dados para formato json
pvl_json = pvl_rj.json()

# criando um dataframe a partir dos dados, na chave de dict items
df_pvl_rj = pd.DataFrame(pvl_json["items"])
df_pvl_rj.head(2)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,22594,Estado,Rio de Janeiro,33,RJ,00000.000000/2031-89,Arquivado por decurso de prazo,17944.000657/2016-95,2016-05-12T03:00:00Z,Operação contratual interna (com garantia da U...,Infraestrutura,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,989210440.0,0,0,25/10/2016
1,10606,Estado,Rio de Janeiro,33,RJ,None,Deferido,17944.001240/2009-11,2009-10-22T02:00:00Z,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Banco do Brasil S/A,Real,20000000.0,1,0,04/11/2009


In [33]:
# 2) Seu código aqui

df_pvl_rj["status"].value_counts()

Encaminhado à PGFN com manifestação técnica favorável    34
Deferido                                                 14
Arquivado a pedido                                        3
Arquivado por decurso de prazo                            1
Name: status, dtype: int64

In [93]:
# 3) Seu código aqui

# cortando uma coluna do dataframe para uma variavel
data_status = df_pvl_rj["data_status"]

# covertendo em str e cortando a partir do indice 6
ano = data_status.str.slice(start=6)
df_pvl_rj["ano_status"] = ano
df_pvl_rj.head(2)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status,ano_status
0,22594,Estado,Rio de Janeiro,33,RJ,00000.000000/2031-89,Arquivado por decurso de prazo,17944.000657/2016-95,2016-05-12T03:00:00Z,Operação contratual interna (com garantia da U...,Infraestrutura,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,989210440.0,0,0,25/10/2016,2016
1,10606,Estado,Rio de Janeiro,33,RJ,None,Deferido,17944.001240/2009-11,2009-10-22T02:00:00Z,Operação contratual interna,"Aquisição de máquinas, equipamentos e veículos",Instituição Financeira Nacional,Banco do Brasil S/A,Real,20000000.0,1,0,04/11/2009,2009


####  2) Melhorando a interação com o API
Observe dois URLs de consultas diferentes, por exemplo o URL utilizado em aula, e o URL feito no exercício anterior. Compare-os e observe as diferenças.

1. Faça uma função em Python que recebe como argumento o UF da consulta e o tipo de interessado (```'Estado'```ou ```Município```), e que devolve os dados da consulta no formato *DataFrame*.
2. Quantas solicitações para o Estado podem ser consultadas para Minas Gerais com *status* em 'Arquivado por decurso de prazo' estão registradas?
3. Qual é o município da Bahia com mais solicitações deferidas?
4. Salve um arquivo .csv com os dados de solicitações da Bahia, com interessado = 'Estado'

In [61]:
#1) Seu código aqui

def consulta_url_pvl(uf: str, interesse: str) -> pd.DataFrame:
    
    # capturando o url, alterando os dados que variam na busca e prevendo inadequacoes
    url = f"http://apidatalake.tesouro.gov.br/ords/sadipem/tt/pvl?uf={uf.upper()}&tipo_interessado={interesse.capitalize()}"
    
    # importando os dados, convertendo em json e montando o dataframe
    r = requests.get(url)
    pvl_json = r.json()
    df = pd.DataFrame(pvl_json["items"])
    
    # retornando o dataframe
    return df

In [92]:
uf = "es"
interesse = "estado"

# chamando a funcao e passando parametros em minusculo
consulta_url_pvl(uf=uf, interesse=interesse).head(2)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,13459,Estado,Espírito Santo,32,ES,None,Deferido,19405.000005/2004-40,2004-02-27T03:00:00Z,Operação contratual interna,Renegociação de dívidas,Instituição Financeira Nacional,Banco Nacional de Desenvolvimento Econômico e ...,Real,268474.0,1,0,19/03/2004
1,23388,Estado,Espírito Santo,32,ES,PVL02.000344/2016-74,Arquivado por decurso de prazo,17944.001165/2016-17,2016-12-19T02:00:00Z,Operação contratual interna (com garantia da U...,Infraestrutura,Instituição Financeira Nacional,Banco do Brasil S/A,Real,250000000.0,0,0,08/08/2017


In [96]:
# 2) Seu código aqui

# captando os dados de mg e armazenando em uma variavel
df_mg_pvl = consulta_url_pvl(uf="mg", interesse="estado")

# criando uma coluna de elementos boleanos atravez de uma operacao logica
arquivado_prazo = df_mg_pvl["status"] == "Arquivado por decurso de prazo"

# contando os valores
df_mg_pvl[arquivado_prazo]["status"].value_counts()

Arquivado por decurso de prazo    1
Name: status, dtype: int64

In [88]:
# 3) Seu código aqui

# consultando os dados para BA e armazenando em uma variavel
df_ba_pvl_municipio = consulta_url_pvl(uf="ba", interesse="município")

# criando uma coluna de boleanos atravez de uma operacao logica
deferido = df_ba_pvl_municipio["status"] == "Deferido"

# contando os valores por municipio
df_ba_pvl_municipio[deferido]["interessado"].value_counts()

Luís Eduardo Magalhães    15
Vitória da Conquista      11
Barreiras                 10
Lauro de Freitas          10
Camaçari                  10
                          ..
Encruzilhada               1
Feira da Mata              1
Ruy Barbosa                1
Ibicuí                     1
Planaltino                 1
Name: interessado, Length: 181, dtype: int64

In [90]:
# 4) Seu código aqui

df_ba_pvl = consulta_url_pvl(uf="ba", interesse="estado")
df_ba_pvl.head(2)

,id_pleito,tipo_interessado,interessado,cod_ibge,uf,num_pvl,status,num_processo,data_protocolo,tipo_operacao,finalidade,tipo_credor,credor,moeda,valor,pvl_assoc_divida,pvl_contradado_credor,data_status
0,10859,Estado,Bahia,29,BA,None,Deferido,17944.001302/2013-71,2013-10-11T03:00:00Z,Operação contratual interna,PAC 2 - Mobilidade Grandes Cidades,Instituição Financeira Nacional,Caixa Econômica Federal,Real,400000000.0,1,0,18/10/2013
1,11514,Estado,Bahia,29,BA,None,Deferido,17944.001466/2007-50,2007-10-26T02:00:00Z,Operação contratual interna,Segurança pública,Instituição Financeira Nacional,Banco do Brasil S/A,Dólar dos EUA,70000000.0,1,0,23/11/2007


In [97]:
# salvando os dados em formato de csv na mesma pagina do notebook
df_ba_pvl.to_csv("solicitacoes_BA")